# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pachino,36.7186,15.0907,25.87,67,0,9.52,IT,1724109157
1,1,saldanha,-33.0117,17.9442,12.59,86,0,3.03,ZA,1724109158
2,2,port elizabeth,-33.9180,25.5701,16.29,63,0,2.06,ZA,1724109161
3,3,belmonte,-15.8631,-38.8828,22.43,86,46,3.51,BR,1724109162
4,4,port-aux-francais,-49.3500,70.2167,3.20,81,54,15.81,TF,1724109163


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

cities_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles= 'OSM',
    frame_width= 800,
    frame_height= 600,
    size= 'Humidity',
    color= 'City'
)
cities_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_condition = city_data_df.loc[
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
].dropna()

# Display sample data
ideal_weather_condition

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,sao vicente,-23.9631,-46.3919,23.94,58,0,2.21,BR,1724109210
147,147,pacific grove,36.6177,-121.9166,21.82,74,0,4.12,US,1724109331
178,178,argo,37.6333,22.7333,25.29,65,0,1.51,GR,1724109367
242,242,novyy turtkul',41.5500,61.0167,21.88,35,0,3.88,UZ,1724109445
320,320,amasra,41.7463,32.3863,21.19,78,0,2.08,TR,1724109541
346,346,mizdah,31.4451,12.9801,24.53,60,0,3.18,LY,1724109571
409,409,alpu,39.7690,30.9606,21.30,68,0,3.37,TR,1724109646
425,425,parsabad,39.6482,47.9174,23.04,68,0,2.47,IR,1724109664
455,455,piracanjuba,-17.3028,-49.0167,24.44,19,0,1.48,BR,1724109701
517,517,ankeny,41.7297,-93.6058,25.87,58,0,0.89,US,1724109776


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,pachino,IT,36.7186,15.0907,67,
1,saldanha,ZA,-33.0117,17.9442,86,
2,port elizabeth,ZA,-33.9180,25.5701,63,
3,belmonte,BR,-15.8631,-38.8828,86,
4,port-aux-francais,TF,-49.3500,70.2167,81,
...,...,...,...,...,...,...
555,rawson,AR,-43.3002,-65.1023,75,
556,salmon arm,CA,50.6998,-119.3024,39,
557,jijiga,ET,9.3500,42.8000,58,
558,baise city,CN,23.8997,106.6133,99,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
api_key = geoapify_key

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon= row["Lng"]
# Add the current city's latitude and longitude to the params dictionary
    filters = f"circle:{lon},{lat},{radius}"
    bias= f"proximity:{lon},{lat}"
    params = {
        "categories": categories,
        "filter": filters,
        "bias": bias,
        "apiKey": api_key,
}

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pachino - nearest hotel: Hotel Celeste
saldanha - nearest hotel: Hoedjesbaai
port elizabeth - nearest hotel: Waterford Hotel
belmonte - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
saint-denis - nearest hotel: Tribe
svetlyy - nearest hotel: Гостиница Спортивная
klyuchi - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
minusinsk - nearest hotel: Саяны
saint-pierre - nearest hotel: Tropic Hotel
edinburgh of the seven seas - nearest hotel: No hotel found
puerto natales - nearest hotel: Dorotea Patagonia Hostel
longyearbyen - nearest hotel: The Vault
avarua - nearest hotel: Paradise Inn
el alamo - nearest hotel: Casarrubios
buala - nearest hotel: No hotel found
nahiyat al fuhud - nearest hotel: No hotel found
gulf shores - nearest hotel: Beachside Resort
albany - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
badger - nearest hotel: Clarion Hotel & Suites
dwarka - nearest hotel: The Dwarika Hotel
gr

,City,Country,Lat,Lng,Humidity,Hotel Name
0,pachino,IT,36.7186,15.0907,67,Hotel Celeste
1,saldanha,ZA,-33.0117,17.9442,86,Hoedjesbaai
2,port elizabeth,ZA,-33.9180,25.5701,63,Waterford Hotel
3,belmonte,BR,-15.8631,-38.8828,86,No hotel found
4,port-aux-francais,TF,-49.3500,70.2167,81,Keravel
...,...,...,...,...,...,...
555,rawson,AR,-43.3002,-65.1023,75,Hotel Deportivo
556,salmon arm,CA,50.6998,-119.3024,39,SureStay Plus Hotel by Best Western Salmon Arm
557,jijiga,ET,9.3500,42.8000,58,Africa Hotel
558,baise city,CN,23.8997,106.6133,99,梦之源大酒店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

hotels_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles= 'OSM',
    frame_width= 800,
    frame_height= 600,
    size= 'Humidity',
    color= 'City',
    hover_cols = ['Hotel Name', 'Country']
)
hotels_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)